In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

c:\Users\vidyu\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-02 16:45:13,678 INFO: Initializing external client
2025-03-02 16:45:13,678 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 16:45:14,997 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215639
Fetching data from 2025-02-01 21:45:13.670219+00:00 to 2025-03-02 20:45:13.670219+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.68s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-01 22:00:00+00:00,2,0
1,2025-02-01 23:00:00+00:00,2,0
2,2025-02-02 00:00:00+00:00,2,0
3,2025-02-02 01:00:00+00:00,2,0
4,2025-02-02 02:00:00+00:00,2,0
...,...,...,...
174440,2025-03-02 16:00:00+00:00,263,93
174441,2025-03-02 17:00:00+00:00,263,114
174442,2025-03-02 18:00:00+00:00,263,114
174443,2025-03-02 19:00:00+00:00,263,93


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 178962
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 178962
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,3,0,0,3,5,1,...,0,0,15,0,2,0,1,0,255,2025-02-19 13:00:00
1,2,1,0,1,1,1,0,1,1,1,...,0,0,1,1,1,0,0,2,218,2025-02-07 15:00:00
2,369,273,249,21,251,348,17,373,286,251,...,468,212,329,232,77,310,138,24,237,2025-02-17 14:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-02-11 05:00:00
4,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,2,0,0,0,56,2025-02-10 17:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,187,2025-02-28 03:00:00
247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,2025-02-07 11:00:00
248,0,2,0,0,4,0,1,0,0,2,...,0,0,0,3,0,1,0,0,71,2025-02-21 11:00:00
249,0,1,1,1,2,0,3,4,0,1,...,1,1,1,1,0,1,0,0,168,2025-02-02 14:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-02 16:46:04,417 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 16:46:04,421 INFO: Initializing external client
2025-03-02 16:46:04,422 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 16:46:05,537 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215639
Fetching data from 2025-02-01 21:46:04.416865+00:00 to 2025-03-02 20:46:04.416865+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.82s) 


In [12]:
current_date

Timestamp('2025-03-02 21:46:04.416865+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-01 22:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,2025-03-01 22:00:00
2,16,38,47,85,75,36,7,1,0,2,...,4,11,12,27,16,16,4,11,4,2025-03-01 22:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,6,2025-03-01 22:00:00
4,3,2,2,1,2,0,1,1,2,1,...,9,2,3,3,4,4,3,2,7,2025-03-01 22:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-01 22:00:00
247,0,1,0,0,1,0,0,0,2,4,...,1,0,1,3,4,1,1,1,260,2025-03-01 22:00:00
248,15,27,18,7,7,5,2,1,3,1,...,42,46,50,57,54,37,23,20,261,2025-03-01 22:00:00
249,27,22,20,7,5,3,3,6,11,30,...,110,99,85,112,103,95,58,26,262,2025-03-01 22:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-02 16:46:38,633 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-02 16:46:38,639 INFO: Initializing external client
2025-03-02 16:46:38,639 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-02 16:46:39,670 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215639


In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,18.0
3,6,0.0
4,7,3.0
...,...,...
246,259,0.0
247,260,1.0
248,261,19.0
249,262,28.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([142,  79, 249, 230,  48, 132, 161, 163, 237, 234])